In [1]:
!pip install prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install plotly==5.10.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 4.2 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


# **National Footprint Accounts CSV file analysis**

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight') # For plots

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset = pd.read_csv("/content/drive/MyDrive/Fullstack/Project data/NFA2018.csv")


In [6]:
dataset.describe(include = "all")

,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
count,87020,85660,87020,87020,87020.000000,87020,6.353000e+04,6.353000e+04,6.353000e+04,6.353000e+04,6.353000e+04,6.353000e+04,8.702000e+04,71410.000000,8.702000e+04
unique,196,193,7,23,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,World,LBR,Africa,Eastern Africa,NaN,BiocapPerCap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,540,540,27690,8770,NaN,8702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,1989.193059,NaN,1.465068e+07,6.324612e+06,1.531334e+07,3.576364e+06,1.311909e+06,2.591459e+07,5.217695e+07,10129.722779,6.305956e+07
std,NaN,NaN,NaN,NaN,15.490923,NaN,1.445766e+08,6.237420e+07,1.798437e+08,3.905889e+07,1.448057e+07,3.448775e+08,5.890184e+08,15646.206882,4.264394e+08
min,NaN,NaN,NaN,NaN,1961.000000,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.150000e-05,115.794000,5.000000e+03
25%,NaN,NaN,NaN,NaN,1976.000000,NaN,3.027605e-01,7.470049e-02,1.508091e-01,3.252735e-02,0.000000e+00,1.946352e-02,1.313131e+00,988.470000,2.001000e+06
50%,NaN,NaN,NaN,NaN,1990.000000,NaN,2.841671e+00,2.526463e+00,3.110603e+00,3.670478e+00,3.247780e-02,1.293047e+00,1.858845e+02,3153.740000,6.632500e+06
75%,NaN,NaN,NaN,NaN,2003.000000,NaN,2.027347e+06,6.580112e+05,1.497303e+06,2.436509e+05,5.794159e+04,9.168465e+05,7.792829e+06,11972.100000,2.056300e+07


In [7]:
dataset.columns

Index(['country', 'ISO alpha-3 code', 'UN_region', 'UN_subregion', 'year',
       'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon', 'total', 'Percapita GDP (2010 USD)',
       'population'],
      dtype='object')

In [8]:
dataset.head()


,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
0,Armenia,ARM,Asia,Western Asia,1992,BiocapPerCap,1.611286e-01,0.135023,0.083836,0.013718,0.033669,0.000000e+00,4.273741e-01,949.033,3449000
1,Armenia,ARM,Asia,Western Asia,1992,BiocapTotGHA,5.558130e+05,465763.337400,289190.662300,47320.224590,116139.598200,0.000000e+00,1.474227e+06,949.033,3449000
2,Armenia,ARM,Asia,Western Asia,1992,EFConsPerCap,3.909225e-01,0.189137,0.000001,0.004138,0.033669,1.112225e+00,1.730092e+00,949.033,3449000
3,Armenia,ARM,Asia,Western Asia,1992,EFConsTotGHA,1.348487e+06,652429.066600,4.327841,14272.803690,116139.598200,3.836620e+06,5.967954e+06,949.033,3449000
4,Armenia,ARM,Asia,Western Asia,1992,EFExportsPerCap,1.124910e-03,0.002283,0.000000,0.000438,0.000000,4.819043e-02,5.203676e-02,949.033,3449000


In [9]:
df = dataset.loc[(dataset['country']=="World") & (dataset['record']=="BiocapPerCap")]
df.head(50)


,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
85700,World,NaN,World,World,1961,BiocapPerCap,0.464565,0.480558,1.787426,0.366753,0.026267,0.0,3.125569,3799.76,3075073000
85710,World,NaN,World,World,1962,BiocapPerCap,0.475082,0.471466,1.752388,0.359066,0.027279,0.0,3.085280,3943.36,3133554000
85720,World,NaN,World,World,1963,BiocapPerCap,0.471512,0.461309,1.714955,0.351070,0.027413,0.0,3.026259,4064.65,3194075000
85730,World,NaN,World,World,1964,BiocapPerCap,0.476642,0.452269,1.681488,0.343549,0.028185,0.0,2.982132,4248.49,3256988000
85740,World,NaN,World,World,1965,BiocapPerCap,0.475159,0.443133,1.648100,0.335574,0.028543,0.0,2.930508,4394.72,3322495000
85750,World,NaN,World,World,1966,BiocapPerCap,0.494471,0.435247,1.615173,0.328713,0.030355,0.0,2.903960,4553.53,3390686000
85760,World,NaN,World,World,1967,BiocapPerCap,0.498460,0.426248,1.581731,0.321066,0.031104,0.0,2.858610,4659.30,3461343000
85770,World,NaN,World,World,1968,BiocapPerCap,0.501645,0.416387,1.543443,0.313847,0.031690,0.0,2.807012,4849.29,3533967000
85780,World,NaN,World,World,1969,BiocapPerCap,0.503505,0.405911,1.507278,0.305276,0.032056,0.0,2.754026,5040.22,3607866000
85790,World,NaN,World,World,1970,BiocapPerCap,0.507482,0.399201,1.474086,0.299711,0.032999,0.0,2.713478,5167.03,3682488000


In [10]:
year_val = df['year'].unique()
print(year_val)
record_val = df['record'].unique()
print(record_val)
country_val = df['country'].unique()
print(country_val)

[1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974
 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988
 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014]
['BiocapPerCap']
['World']


 # **RECORD SEPARATION**

In [11]:
new_data = pd.DataFrame()
line_index =  0
for i in country_val:
    for j in year_val:
        data = dataset.loc[(dataset['country'] == i) & (dataset['year'] == j), :].reset_index(drop=True)
        if data.shape[0] > 0:
            data_clean = pd.pivot_table(data, values='total', columns= 'record')
            new_data = new_data.append(data_clean, ignore_index=True, verify_integrity=False)

            new_data.loc[line_index, 'country'] = i
            new_data.loc[line_index, 'year'] = j
            new_data.loc[line_index, 'population'] = data.loc[0,'population']
            new_data.loc[line_index, 'Percapita GDP (2010 USD)'] = data.loc[0,'Percapita GDP (2010 USD)']

            line_index += 1
        
new_data.head()

record,BiocapPerCap,BiocapTotGHA,EFConsPerCap,EFConsTotGHA,EFExportsPerCap,EFExportsTotGHA,EFImportsPerCap,EFImportsTotGHA,EFProdPerCap,EFProdTotGHA,country,year,population,Percapita GDP (2010 USD)
0,3.125569,9.611344e+09,2.287829,7.035243e+09,0.160294,492915450.9,0.153878,473185309.1,2.287829,7.035243e+09,World,1961.0,3.075073e+09,3799.76
1,3.085280,9.667879e+09,2.318771,7.265996e+09,0.163553,512500982.8,0.154483,484079134.9,2.318771,7.265996e+09,World,1962.0,3.133554e+09,3943.36
2,3.026259,9.666112e+09,2.363952,7.550641e+09,0.169866,542564357.1,0.161793,516778701.1,2.363952,7.550641e+09,World,1963.0,3.194075e+09,4064.65
3,2.982132,9.712774e+09,2.414695,7.864634e+09,0.172577,562081259.5,0.163685,533121334.0,2.414695,7.864634e+09,World,1964.0,3.256988e+09,4248.49
4,2.930508,9.736612e+09,2.454496,8.155050e+09,0.173341,575925562.7,0.166358,552724395.2,2.454496,8.155050e+09,World,1965.0,3.322495e+09,4394.72


In [12]:
nfa = new_data.loc[new_data['year'] >= 1961, :]
nfa.head()

record,BiocapPerCap,BiocapTotGHA,EFConsPerCap,EFConsTotGHA,EFExportsPerCap,EFExportsTotGHA,EFImportsPerCap,EFImportsTotGHA,EFProdPerCap,EFProdTotGHA,country,year,population,Percapita GDP (2010 USD)
0,3.125569,9.611344e+09,2.287829,7.035243e+09,0.160294,492915450.9,0.153878,473185309.1,2.287829,7.035243e+09,World,1961.0,3.075073e+09,3799.76
1,3.085280,9.667879e+09,2.318771,7.265996e+09,0.163553,512500982.8,0.154483,484079134.9,2.318771,7.265996e+09,World,1962.0,3.133554e+09,3943.36
2,3.026259,9.666112e+09,2.363952,7.550641e+09,0.169866,542564357.1,0.161793,516778701.1,2.363952,7.550641e+09,World,1963.0,3.194075e+09,4064.65
3,2.982132,9.712774e+09,2.414695,7.864634e+09,0.172577,562081259.5,0.163685,533121334.0,2.414695,7.864634e+09,World,1964.0,3.256988e+09,4248.49
4,2.930508,9.736612e+09,2.454496,8.155050e+09,0.173341,575925562.7,0.166358,552724395.2,2.454496,8.155050e+09,World,1965.0,3.322495e+09,4394.72


In [13]:
#nfa = new_data.loc[new_data['year'] <= 2005, :]

In [14]:
nfa['year'] = nfa['year'].astype(int)
nfa.tail()

record,BiocapPerCap,BiocapTotGHA,EFConsPerCap,EFConsTotGHA,EFExportsPerCap,EFExportsTotGHA,EFImportsPerCap,EFImportsTotGHA,EFProdPerCap,EFProdTotGHA,country,year,population,Percapita GDP (2010 USD)
49,1.722745,1.193816e+10,2.866199,1.986197e+10,0.299599,2.076137e+09,0.303925,2.106121e+09,2.866199,1.986197e+10,World,2010,6.929725e+09,9516.35
50,1.707697,1.197681e+10,2.887903,2.025410e+10,0.298639,2.094485e+09,0.303304,2.127201e+09,2.887903,2.025410e+10,World,2011,7.013427e+09,9703.32
51,1.679217,1.191824e+10,2.836151,2.012958e+10,0.306093,2.172493e+09,0.310984,2.207211e+09,2.836151,2.012958e+10,World,2012,7.097500e+09,9821.90
52,1.690722,1.214229e+10,2.866024,2.058297e+10,0.309697,2.224160e+09,0.312800,2.246444e+09,2.866024,2.058297e+10,World,2013,7.181715e+09,9960.54
53,1.682043,1.222137e+10,2.835469,2.060191e+10,0.308779,2.243524e+09,0.311697,2.264723e+09,2.835469,2.060191e+10,World,2014,7.265786e+09,10119.50


In [15]:
new_data.columns

Index(['BiocapPerCap', 'BiocapTotGHA', 'EFConsPerCap', 'EFConsTotGHA',
       'EFExportsPerCap', 'EFExportsTotGHA', 'EFImportsPerCap',
       'EFImportsTotGHA', 'EFProdPerCap', 'EFProdTotGHA', 'country', 'year',
       'population', 'Percapita GDP (2010 USD)'],
      dtype='object', name='record')

In [16]:
nfa['year_clean'] = pd.to_datetime(nfa['year'], format='%Y')
nfa.tail()

record,BiocapPerCap,BiocapTotGHA,EFConsPerCap,EFConsTotGHA,EFExportsPerCap,EFExportsTotGHA,EFImportsPerCap,EFImportsTotGHA,EFProdPerCap,EFProdTotGHA,country,year,population,Percapita GDP (2010 USD),year_clean
49,1.722745,1.193816e+10,2.866199,1.986197e+10,0.299599,2.076137e+09,0.303925,2.106121e+09,2.866199,1.986197e+10,World,2010,6.929725e+09,9516.35,2010-01-01
50,1.707697,1.197681e+10,2.887903,2.025410e+10,0.298639,2.094485e+09,0.303304,2.127201e+09,2.887903,2.025410e+10,World,2011,7.013427e+09,9703.32,2011-01-01
51,1.679217,1.191824e+10,2.836151,2.012958e+10,0.306093,2.172493e+09,0.310984,2.207211e+09,2.836151,2.012958e+10,World,2012,7.097500e+09,9821.90,2012-01-01
52,1.690722,1.214229e+10,2.866024,2.058297e+10,0.309697,2.224160e+09,0.312800,2.246444e+09,2.866024,2.058297e+10,World,2013,7.181715e+09,9960.54,2013-01-01
53,1.682043,1.222137e+10,2.835469,2.060191e+10,0.308779,2.243524e+09,0.311697,2.264723e+09,2.835469,2.060191e+10,World,2014,7.265786e+09,10119.50,2014-01-01


In [17]:
nfa['year_clean']= pd.date_range(start='1/1/1961', periods=54, freq='Y')
nfa.head()


record,BiocapPerCap,BiocapTotGHA,EFConsPerCap,EFConsTotGHA,EFExportsPerCap,EFExportsTotGHA,EFImportsPerCap,EFImportsTotGHA,EFProdPerCap,EFProdTotGHA,country,year,population,Percapita GDP (2010 USD),year_clean
0,3.125569,9.611344e+09,2.287829,7.035243e+09,0.160294,492915450.9,0.153878,473185309.1,2.287829,7.035243e+09,World,1961,3.075073e+09,3799.76,1961-12-31
1,3.085280,9.667879e+09,2.318771,7.265996e+09,0.163553,512500982.8,0.154483,484079134.9,2.318771,7.265996e+09,World,1962,3.133554e+09,3943.36,1962-12-31
2,3.026259,9.666112e+09,2.363952,7.550641e+09,0.169866,542564357.1,0.161793,516778701.1,2.363952,7.550641e+09,World,1963,3.194075e+09,4064.65,1963-12-31
3,2.982132,9.712774e+09,2.414695,7.864634e+09,0.172577,562081259.5,0.163685,533121334.0,2.414695,7.864634e+09,World,1964,3.256988e+09,4248.49,1964-12-31
4,2.930508,9.736612e+09,2.454496,8.155050e+09,0.173341,575925562.7,0.166358,552724395.2,2.454496,8.155050e+09,World,1965,3.322495e+09,4394.72,1965-12-31


# **PREDICTION WITH FB PROPHET**



# **Prediction of EFCONS/CAP**


In [18]:
p_df = pd.DataFrame({
    "ds":nfa['year_clean'],
    "y": nfa['BiocapPerCap']
}).reset_index(drop=True)

p_df.head()

,ds,y
0,1961-12-31,3.125569
1,1962-12-31,3.085280
2,1963-12-31,3.026259
3,1964-12-31,2.982132
4,1965-12-31,2.930508


In [19]:
m = Prophet()
m.fit(p_df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw6m1wfz/4ikr59zf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw6m1wfz/9axe5oav.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=46577', 'data', 'file=/tmp/tmplw6m1wfz/4ikr59zf.json', 'init=/tmp/tmplw6m1wfz/9axe5oav.json', 'output', 'file=/tmp/tmplw6m1wfz/prophet_model7yk3180p/prophet_model-20221026112001.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:20:01 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:20:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [20]:
future = m.make_future_dataframe(freq="A", periods = 11 )
future

,ds
0,1961-12-31
1,1962-12-31
2,1963-12-31
3,1964-12-31
4,1965-12-31
...,...
60,2021-12-31
61,2022-12-31
62,2023-12-31
63,2024-12-31


In [21]:
forecast = m.predict(future)
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
60,2021-12-31,-1.635013,1.553264,1.592123,-1.650400,-1.618819,3.207004,3.207004,3.207004,3.207004,3.207004,3.207004,0.0,0.0,0.0,1.571991
61,2022-12-31,-1.649171,1.533290,1.579566,-1.667484,-1.628855,3.206507,3.206507,3.206507,3.206507,3.206507,3.206507,0.0,0.0,0.0,1.557336
62,2023-12-31,-1.663329,1.510357,1.562485,-1.685540,-1.639192,3.200305,3.200305,3.200305,3.200305,3.200305,3.200305,0.0,0.0,0.0,1.536976
63,2024-12-31,-1.677526,1.495808,1.553034,-1.703304,-1.649301,3.202054,3.202054,3.202054,3.202054,3.202054,3.202054,0.0,0.0,0.0,1.524528
64,2025-12-31,-1.691684,1.480986,1.548594,-1.722069,-1.659554,3.207004,3.207004,3.207004,3.207004,3.207004,3.207004,0.0,0.0,0.0,1.515321


In [22]:
from prophet.plot import plot_plotly, plot_components_plotly
#import plotly.offline as py
#py.init_notebook_mode(connected=True)

fig = plot_plotly(m, forecast)  # This returns a plotly Figure
fig.show(renderer="colab")

# **Prediction of EFCONS/CAP**

In [23]:
p_df1 = pd.DataFrame({
    "ds":nfa['year_clean'],
    "y": nfa['EFConsPerCap']
}).reset_index(drop=True)

p_df1.tail()

,ds,y
49,2010-12-31,2.866199
50,2011-12-31,2.887903
51,2012-12-31,2.836151
52,2013-12-31,2.866024
53,2014-12-31,2.835469


In [24]:
model = Prophet()
model.fit(p_df1)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw6m1wfz/yq2d3hlg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw6m1wfz/en27x1_b.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97688', 'data', 'file=/tmp/tmplw6m1wfz/yq2d3hlg.json', 'init=/tmp/tmplw6m1wfz/en27x1_b.json', 'output', 'file=/tmp/tmplw6m1wfz/prophet_modelrx5vgg3r/prophet_model-20221026112003.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:20:03 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:20:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [25]:
future1 = model.make_future_dataframe(freq="A", periods = 10 )
future1

,ds
0,1961-12-31
1,1962-12-31
2,1963-12-31
3,1964-12-31
4,1965-12-31
...,...
59,2020-12-31
60,2021-12-31
61,2022-12-31
62,2023-12-31


In [26]:
forecast1 = model.predict(future1)
forecast1.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1961-12-31,0.297110,2.240561,2.366464,0.297110,0.297110,2.004838,2.004838,2.004838,2.004838,2.004838,2.004838,0.0,0.0,0.0,2.301948
1,1962-12-31,0.338829,2.271995,2.405923,0.338829,0.338829,2.002958,2.002958,2.002958,2.002958,2.002958,2.002958,0.0,0.0,0.0,2.341787
2,1963-12-31,0.380547,2.322511,2.446279,0.380547,0.380547,2.004637,2.004637,2.004637,2.004637,2.004637,2.004637,0.0,0.0,0.0,2.385184
3,1964-12-31,0.422380,2.368357,2.492821,0.422380,0.422380,2.010279,2.010279,2.010279,2.010279,2.010279,2.010279,0.0,0.0,0.0,2.432659
4,1965-12-31,0.464098,2.408325,2.535990,0.464098,0.464098,2.004838,2.004838,2.004838,2.004838,2.004838,2.004838,0.0,0.0,0.0,2.468936


In [27]:
from prophet.plot import plot_plotly
import plotly.offline as py
import matplotlib.pyplot as plt
py.init_notebook_mode()

fig = plot_plotly(model,forecast1)  # This returns a plotly Figure

fig.show(renderer="colab")

Output hidden; open in https://colab.research.google.com to view.

# **Multi-variate forecasting** 
(Check our current values from 2005 to 2014)




In [28]:
bio = pd.DataFrame({
    "ds":nfa['year_clean'],
    "y": nfa['BiocapPerCap']
}).reset_index(drop=True)

bio.head()

,ds,y
0,1961-12-31,3.125569
1,1962-12-31,3.085280
2,1963-12-31,3.026259
3,1964-12-31,2.982132
4,1965-12-31,2.930508


In [29]:
split_date = '2004-12-31'
bio_train = bio.loc[bio['ds'] <= split_date]
bio_test = bio.loc[bio['ds']> split_date]


In [30]:
#Y_train
y_train = bio_train.iloc[:,-1]
y_train.values

array([3.12556902, 3.08528007, 3.02625867, 2.98213209, 2.9305078 ,
       2.90395986, 2.85860965, 2.8070123 , 2.75402616, 2.71347808,
       2.68192161, 2.61756968, 2.59593363, 2.53163813, 2.49097958,
       2.46450664, 2.42370775, 2.41616959, 2.36973025, 2.32819387,
       2.31019494, 2.29329283, 2.23834625, 2.23793715, 2.21576706,
       2.18392108, 2.14865832, 2.09194549, 2.08769036, 2.08221332,
       2.03283793, 2.0345038 , 2.00105082, 1.9890855 , 1.94816174,
       1.95112314, 1.93356498, 1.92013928, 1.89995591, 1.87445803,
       1.86142455, 1.8376373 , 1.81083763, 1.82624185])

In [31]:
#Y_test
y_test = bio_test.iloc[:,-1]
y_test.values

array([1.79323608, 1.77496571, 1.76015266, 1.76698265, 1.73696263,
       1.72274472, 1.70769714, 1.67921684, 1.6907218 , 1.6820434 ])

In [32]:
# Setup and train model and fit
model = Prophet()
model.fit(bio_train)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw6m1wfz/reo2mcfj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw6m1wfz/4i5zagpe.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=24414', 'data', 'file=/tmp/tmplw6m1wfz/reo2mcfj.json', 'init=/tmp/tmplw6m1wfz/4i5zagpe.json', 'output', 'file=/tmp/tmplw6m1wfz/prophet_modeluf4tqjna/prophet_model-20221026112005.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:20:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:20:07 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [33]:
#Y_test_pred
Y_test_pred = forecast["yhat"].values
Y_test_pred

array([3.12407235, 3.07867605, 3.02757518, 2.98430131, 2.94435288,
       2.89895658, 2.84785571, 2.80458183, 2.76463341, 2.71924299,
       2.66814801, 2.62488004, 2.5849375 , 2.53954776, 2.49539503,
       2.45908833, 2.43434562, 2.40415503, 2.3682615 , 2.34026187,
       2.31554596, 2.28692713, 2.25260373, 2.2261533 , 2.20298234,
       2.17436351, 2.14004011, 2.11358968, 2.09547255, 2.07190754,
       2.04263875, 2.02343482, 2.00749057, 1.98728383, 1.96137252,
       1.94545359, 1.93278536, 1.91466926, 1.89084859, 1.87492966,
       1.86226144, 1.84414769, 1.82032938, 1.80788089, 1.79867363,
       1.78401848, 1.76365877, 1.75121028, 1.74200302, 1.72734787,
       1.70698816, 1.69453967, 1.68533241, 1.67067726, 1.65031755,
       1.63786906, 1.6286618 , 1.61400665, 1.59364694, 1.58119845,
       1.57199118, 1.55733604, 1.53697633, 1.52452784, 1.51532057])

In [34]:
#Y_test_pred
y_train_pred = model.predict(bio_train)
y_train_pred2= y_train_pred['yhat'].values
y_train_pred2

array([3.12386169, 3.07866163, 3.02794762, 2.98404689, 2.94413394,
       2.89896061, 2.84827333, 2.8043994 , 2.76450894, 2.71955554,
       2.66908818, 2.62543479, 2.58579335, 2.54088976, 2.4904722 ,
       2.46085699, 2.43519275, 2.40428757, 2.36786843, 2.33831491,
       2.31522952, 2.28688005, 2.25301663, 2.22601266, 2.20294266,
       2.17456967, 2.14068272, 2.11365516, 2.09540317, 2.07187173,
       2.04286656, 2.02336096, 2.0078047 , 1.98696898, 1.96066824,
       1.94466435, 1.93256438, 1.91518495, 1.89229155, 1.87628767,
       1.86418769, 1.84680826, 1.82391487, 1.80791098])

In [35]:
forecast = model.predict(bio_test)

In [36]:
# Viz
fig = plot_plotly(model, forecast)  # This returns a plotly Figure
fig.show(renderer="colab")


In [37]:
future2 = m.make_future_dataframe(freq= "Y",periods=10)
future2

,ds
0,1961-12-31
1,1962-12-31
2,1963-12-31
3,1964-12-31
4,1965-12-31
...,...
59,2020-12-31
60,2021-12-31
61,2022-12-31
62,2023-12-31


In [38]:
forecast2 = model.predict(future2)
forecast2.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1961-12-31,-0.074550,3.111508,3.136028,-0.074550,-0.074550,3.198412,3.198412,3.198412,3.198412,3.198412,3.198412,0.0,0.0,0.0,3.123862
1,1962-12-31,-0.119453,3.066612,3.090120,-0.119453,-0.119453,3.198115,3.198115,3.198115,3.198115,3.198115,3.198115,0.0,0.0,0.0,3.078662
2,1963-12-31,-0.164356,3.015416,3.039251,-0.164356,-0.164356,3.192304,3.192304,3.192304,3.192304,3.192304,3.192304,0.0,0.0,0.0,3.027948
3,1964-12-31,-0.209382,2.972205,2.995974,-0.209382,-0.209382,3.193429,3.193429,3.193429,3.193429,3.193429,3.193429,0.0,0.0,0.0,2.984047
4,1965-12-31,-0.254278,2.932869,2.956707,-0.254278,-0.254278,3.198412,3.198412,3.198412,3.198412,3.198412,3.198412,0.0,0.0,0.0,2.944134
